In [7]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.utils import to_categorical
from glob import glob
import numpy as np

In [2]:
#### CONFIG ####
batch_size = 150
with_color = True
height, width = 64,64
early_stop_step = 5
use_multiprocess = True
num_of_worker=3

In [3]:
if with_color:
    my_color_mode = 'rgb'
    my_input_shape = (height,width,3)
else:
    my_color_mode = 'grayscale'
    my_input_shape =(height,width,1)
    
# this is the configuration for validation set 
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# For testing only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator, will read pictures found in subfolers of 'data/train'
train_generator = train_datagen.flow_from_directory(
        './train/',  # this is the target directory
        target_size=my_input_shape[:-1],  # all images will be resized to 150x150
        color_mode = my_color_mode, # rgb, grayscale
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a generator, will read pictures found in subfolers of 'data/train'
validation_generator = test_datagen.flow_from_directory(
        './validation/',  # this is the target directory
        target_size=my_input_shape[:-1],  # all images will be resized to 150x150
        color_mode = my_color_mode, # rgb, grayscale
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [4]:
#cifar
model_v2 = Sequential()
model_v2.add(Conv2D(32,kernel_size=(5, 5), activation= 'relu',  input_shape=my_input_shape))
model_v2.add(Conv2D(32,kernel_size=(5, 5), activation= 'relu'))
model_v2.add(MaxPooling2D(pool_size=(2, 2),strides=2))
model_v2.add(Dropout(0.2))

model_v2.add(Conv2D(16,kernel_size=(5, 5), activation= 'relu'))
model_v2.add(Conv2D(16,kernel_size=(5, 5), activation= 'relu'))
model_v2.add(MaxPooling2D(pool_size=(2, 2),strides=2))
model_v2.add(Dropout(0.2))

model_v2.add(Flatten())

model_v2.add(Dense(1024,activation='relu'))

model_v2.add(Dropout(0.2))
model_v2.add(Dense(1))
model_v2.add(Activation('sigmoid'))

In [5]:
model_v2.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [9]:
#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', # cosa monitorare
                               mode='auto',
                               verbose =1,
                               patience=early_stop_step
                              ) # fermati in anticipo

tbCallBack = TensorBoard(log_dir='./Graph',
                         histogram_freq=0, write_graph=True, write_images=True)
                        # tensorboard --logdir ./Graph

checkpoint = ModelCheckpoint(# Save the model after every epoch  weights.{epoch:02d}-{val_loss:.2f}.hdf5
                            './CheckPoint/checkpoint.hdf5',
                             monitor='val_acc', # scegli sulla base dell'accuracy sul validation
                             verbose=1, # 0 silente, 1 verboso
                             save_best_only=True, # il modello momentaneamente migliore 
                                                  # sovrascrive quello che era migliore
                             mode='auto' # puoi scegliere max per l'accuracy
                                         # o min per la loss, con auto capisce lui cos'è melgio
                            )

# Train model on dataset
model_v2.fit_generator(generator=train_generator,
                    validation_data=validation_generator,
                    epochs=50, # numero di epoche, potrebbe essere nullo visto che c'è early_stopping
                    callbacks =  [early_stopping, tbCallBack, checkpoint],
                    use_multiprocessing= use_multiprocess,
                    workers=num_of_worker,
                    shuffle=True # dopo ogni epoca ricampiona il dato
                    )

Epoch 1/50
134/134 [==============================] - 580s 4s/step - loss: 0.6654 - acc: 0.6056 - val_loss: 0.6226 - val_acc: 0.6552

Epoch 00001: val_acc improved from -inf to 0.65520, saving model to ./CheckPoint/weights.01-0.62.hdf5
Epoch 2/50
134/134 [==============================] - 589s 4s/step - loss: 0.6353 - acc: 0.6433 - val_loss: 0.5974 - val_acc: 0.6836

Epoch 00002: val_acc improved from 0.65520 to 0.68360, saving model to ./CheckPoint/weights.02-0.60.hdf5
Epoch 3/50
 11/134 [=>............................] - ETA: 8:55 - loss: 0.6099 - acc: 0.6788

Process ForkPoolWorker-30:
Process ForkPoolWorker-29:
Process ForkPoolWorker-27:
Process ForkPoolWorker-28:
Process ForkPoolWorker-25:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/silvio_saglimbeni/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/silvio_saglimbeni/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/silvio_saglimbeni/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/silvio_saglimbeni/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/silvio_saglimbeni/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/silvio_saglimbeni/anaconda3/lib/python3.6/multipro

KeyboardInterrupt: 

In [ ]:
model_v2.summary()

In [ ]:
model_v2.save_weights('second_try.h5')  